In [1]:
import mujoco
import mediapy as media
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
clear_output()

## Algorithms

<u>Ted</u>

``` C
// RK4 tableau
const mjtNum RK4_A[9] = {
  0.5,    0,      0,
  0,      0.5,    0,
  0,      0,      1
};

const mjtNum RK4_B[4] = {
  1.0/6.0, 1.0/3.0, 1.0/3.0, 1.0/6.0
};


// Runge Kutta explicit order-N integrator
//  (A,B) is the tableau, C is set to row_sum(A)
void mj_RungeKutta(const mjModel* m, mjData* d, int N) {
  int nv = m->nv, nq = m->nq, na = m->na;
  mjtNum h = m->opt.timestep, time = d->time;
  mjtNum C[9], T[9], *X[10], *F[10], *dX;
  const mjtNum* A = (N == 4 ? RK4_A : 0);
  const mjtNum* B = (N == 4 ? RK4_B : 0);

  // check order
  if (!A) {
    mjERROR("supported RK orders: N=4");
  }

  // allocate space for intermediate solutions
  mj_markStack(d);
  dX = mjSTACKALLOC(d, 2*nv+na, mjtNum);
  for (int i=0; i < N; i++) {
    X[i] = mjSTACKALLOC(d, nq+nv+na, mjtNum);
    F[i] = mjSTACKALLOC(d, nv+na, mjtNum);
  }

  // precompute C and T;  C,T,A have size (N-1)
  for (int i=1; i < N; i++) {
    // C(i) = sum_j A(i,j)
    C[i-1] = 0;
    for (int j=0; j < i; j++) {
      C[i-1] += A[(i-1)*(N-1)+j];
    }

    // compute T
    T[i-1] = d->time + C[i-1]*h;
  }

  // init X[0], F[0]; mj_forward() was already called
  mju_copy(X[0], d->qpos, nq);
  mju_copy(X[0]+nq, d->qvel, nv);
  mju_copy(F[0], d->qacc, nv);
  if (na) {
    mju_copy(X[0]+nq+nv, d->act, na);
    mju_copy(F[0]+nv, d->act_dot, na);
  }

  // compute the remaining X[i], F[i]
  for (int i=1; i < N; i++) {
    // compute dX
    mju_zero(dX, 2*nv+na);
    for (int j=0; j < i; j++) {
      mju_addToScl(dX, X[j]+nq, A[(i-1)*(N-1)+j], nv);
      mju_addToScl(dX+nv, F[j], A[(i-1)*(N-1)+j], nv+na);
    }

    // compute X[i] = X[0] '+' dX
    mju_copy(X[i], X[0], nq+nv+na);
    mj_integratePos(m, X[i], dX, h);
    mju_addToScl(X[i]+nq, dX+nv, h, nv+na);

    // set X[i], T[i-1] in mjData
    mju_copy(d->qpos, X[i], nq);
    mju_copy(d->qvel, X[i]+nq, nv);
    if (na) {
      mju_copy(d->act, X[i]+nq+nv, na);
    }
    d->time = T[i-1];

    // evaluate F[i]
    mj_forwardSkip(m, d, mjSTAGE_NONE, 1);  // 1: do not recompute sensors and energy
    mju_copy(F[i], d->qacc, nv);
    if (na) {
      mju_copy(F[i]+nv, d->act_dot, na);
    }
  }

  // compute dX for final update (using B instead of A)
  mju_zero(dX, 2*nv+na);
  for (int j=0; j < N; j++) {
    mju_addToScl(dX, X[j]+nq, B[j], nv);
    mju_addToScl(dX+nv, F[j], B[j], nv+na);
  }

  // reset state and time
  d->time = time;
  mju_copy(d->qpos, X[0], nq);
  mju_copy(d->qvel, X[0]+nq, nv);
  mju_copy(d->act, X[0]+nq+nv, na);

  // advance state and time
  mj_advance(m, d, dX+2*nv, dX+nv, dX);

  mj_freeStack(d);
}
```

All integrators are called from `mj_step()` which selects the appropriate integrator.

The `mj_RungeKutta()` function implements a classical 4th-order Runge-Kutta method. The function begins by defining the RK4 coefficients and allocating space for intermediate calculations. RK4 computes 4 approximations for the derivative at different points: at the beginning of the timestep, two midpoints, and the end of the timestep. The final update is a weighted average of these four slopes/steps using the weights defined in RK4_B.

For each stage, the function calculates the intermediate state by advancing the initial state. For each of the 4 points it sets the next state in `mjData` and calls `mj_forwardSkip()` (this function allows to skip certain computational steps and sensor calculations) to evaluate the system at this state and record the resulting derivatives. Finally, in the last loop it computes the weighted sum of these derivatives and stores them in the `dX` state array. Then it resets the state to the initial values from the current invocation, and lastly calls `mj_advance()` with these weighted accelerations and velocities to advance the state of the simulation.

``` C
// fully implicit in velocity, possibly skipping factorization
void mj_implicitSkip(const mjModel* m, mjData* d, int skipfactor) {
  TM_START;
  int nv = m->nv, nM = m->nM, nD = m->nD;

  mj_markStack(d);
  mjtNum* qfrc = mjSTACKALLOC(d, nv, mjtNum);
  mjtNum* qacc = mjSTACKALLOC(d, nv, mjtNum);

  // set qfrc = qfrc_smooth + qfrc_constraint
  mju_add(qfrc, d->qfrc_smooth, d->qfrc_constraint, nv);

  // IMPLICIT
  if (m->opt.integrator == mjINT_IMPLICIT) {
    if (!skipfactor) {
      // compute analytical derivative qDeriv
      mjd_smooth_vel(m, d, /* flg_bias = */ 1);

      // set qLU = qM
      for (int i=0; i < nD; i++) {
        d->qLU[i] = d->qM[d->mapM2D[i]];
      }

      // set qLU = qM - dt*qDeriv
      mju_addToScl(d->qLU, d->qDeriv, -m->opt.timestep, m->nD);

      // factorize qLU
      int* scratch = mjSTACKALLOC(d, nv, int);
      mju_factorLUSparse(d->qLU, nv, scratch, d->D_rownnz, d->D_rowadr, d->D_colind);
    }

    // solve for qacc: (qM - dt*qDeriv) * qacc = qfrc
    mju_solveLUSparse(qacc, d->qLU, qfrc, nv, d->D_rownnz, d->D_rowadr, d->D_diag, d->D_colind);
  }

  // IMPLICITFAST
  else if (m->opt.integrator == mjINT_IMPLICITFAST) {
    if (!skipfactor) {
      // compute analytical derivative qDeriv; skip rne derivative
      mjd_smooth_vel(m, d, /* flg_bias = */ 0);

      // modified mass matrix MhB = qDeriv[Lower]
      mjtNum* MhB = mjSTACKALLOC(d, nM, mjtNum);
      for (int i=0; i < nM; i++) {
        MhB[i] = d->qDeriv[d->mapD2M[i]];
      }

      // set MhB = M - dt*qDeriv
      mju_addScl(MhB, d->qM, MhB, -m->opt.timestep, nM);

      // copy into qH
      for (int i=0; i < nM; i++) {
        d->qH[i] = MhB[d->mapM2M[i]];
      }

      // factorize in-place
      mj_factorI(d->qH, d->qHDiagInv, nv, d->M_rownnz, d->M_rowadr, m->dof_simplenum, d->M_colind);
    }

    // solve for qacc: (qM - dt*qDeriv) * qacc = qfrc
    mju_copy(qacc, qfrc, nv);
    mj_solveLD(qacc, d->qH, d->qHDiagInv, nv, 1,
               d->M_rownnz, d->M_rowadr, m->dof_simplenum, d->M_colind);

  } else {
    mjERROR("integrator must be implicit or implicitfast");
  }

  // advance state and time
  mj_advance(m, d, d->act_dot, qacc, NULL);

  mj_freeStack(d);

  TM_END(mjTIMER_ADVANCE);

  // computed by mj_implicit/mj_derivative
  mjtNum* qDeriv;            // d (passive + actuator - bias) / d qvel           (nD x 1)

  // computed by mj_implicit/mju_factorLUSparse
  mjtNum* qLU;               // sparse LU of (qM - dt*qDeriv)                    (nD x 1)
```

What is implicit?

When we talk about numerical integration for physics simulations, we're trying to advance the state of a dynamic system from the current time step to the next. Explicit methods, like Euler or RK4, use information that's already known (current position, velocity, and forces), calculate accelerations at the current state and use those accelerations to directly predict where the system will be in the future.

In implicit methods, we set up an equation where the future state is the unknown. This equation includes how the forces at the future state affect the motion. We solve this equation to find the state that makes the dynamics consistent. Explicit methods are like predicting where a ball will be after throwing it, and implicit methods are like figuring out where you need to catch the ball so that all the physics equations are satisfied.

Looking at the implicit integrator, first the code computes qfrc as the sum of smooth (net unconstrained) forces and constraint forces from mData. Then, it calls `mjd_smooth_vel()` that calculates the partial derivative of the dynamics with respect to velocity, which captures how small changes in velocity affect the forces and resulting accelerations in the system. 

The implicit integrator then sets up the implicit system matrix that will be used to solve for the acceleration with a call to `mju_factorLUSparse()`. This matrix represents how the future state depends on both the current state and the unknown future accelerations. Then, the call to mju_solveLUSparse() solves the matrix. The implicit integrator uses LU factorization which is a decomposition technique to break down a matrix into the product of a lower triangular (L) and an upper triangular (U) matrix. MuJoCo optimizes this by only storing and operating on non-zero elements which helps with the performance when dealing with large, sparse matrices in physics simulations.

Compared to the implicit integrator, the implicitfast integrator calls `mjd_smooth_vel()` with `flag_bias=0` (which subtracts `d qfrc_bias / d qvel from qDeriv` which ignores bias forces). Unlike the implicit, implicitfast uses a special optimized factorization and a specialized solver to advantage of MuJoCo's matrix structure. The implicitfast method uses a simpler derivative approximation and optimized factorization and solver implementations which makes it a good default choice for simulations that benefit from implicit integration (stiff systems, fast-changing forces).

Finally, in both implicit and implicitfast cases the function calls `mj_advance()` to advance the simulation state.

## Example

<u>Grayson</u>

The example that we examined is the arm26.xml or 2-link 6-muscle arm. As the name suggests, the arm has 2 joints that the parts pivot about and 6 muscles or actuators that can apply force to the body. Both joints are hinge joints which can only rotate about one axis, which gives them both 1 degree of freedom. Together they constitute a 2 degree of freedom robot. The arm can be used by applying force to the muscles. The muscles are attached to sites that are either fixed in the environment or attached to one of the “bones”.

The motion of the arm is calculated by integrating the velocity of arm that is accelerated by the force of the muscle minus the dampening factor. In our example the integrator is the default Euler. During the first 2 seconds the flexion muscles are activated. The damping factor acts against motion proportional to the velocity of the object to simulate air resistance or friction. Without damping the energy of the system would be conserved and the arm would keep swinging as the arm attempts to return to its relaxed state. 

In [2]:
# Grayson

xml = """
<mujoco model="2-link 6-muscle arm">
  <!-- system solved every 0.005 seconds, 50 iterations per times step-->
  <!-- newtonian solver with tolerance 1e-10 -->
  <option timestep="0.005" iterations="50" solver="Newton" tolerance="1e-10"/>

  <visual>
  <!-- gray haze is added to the environment -->
    <rgba haze=".3 .3 .3 1"/>
  </visual>

  <default>
    <!-- Sets default properties for joints. Joint type hinge rotates about 1 axis -->
    <!-- Position at (0,0,0), Axis rotates about Z or vertical axis, range of motion is restricted from 0 to 120 degrees -->
    <!-- Dampling is provides resistance proportional to velocity -->
    <joint type="hinge" pos="0 0 0" axis="0 0 1" limited="true" range="0 120" damping="0.1"/>
    <!-- Muscle energy is limited on a scale of 0 to 1 -->
    <muscle ctrllimited="true" ctrlrange="0 1"/>
  </default>

  <asset>
    <!-- sets sky box texture that transitions from light gray to black -->
    <texture type="skybox" builtin="gradient" rgb1="0.6 0.6 0.6" rgb2="0 0 0" width="512" height="512"/>
    <!-- sets ground plane texture with checkered gray and darker gray tiles with a light cross through the origin -->
    <texture name="texplane" type="2d" builtin="checker" rgb1=".25 .25 .25" rgb2=".3 .3 .3" width="512" height="512" mark="cross" markrgb=".8 .8 .8"/>
    <!-- Adds reflective property to plane -->
    <material name="matplane" reflectance="0.3" texture="texplane" texrepeat="1 1" texuniform="true"/>
  </asset>

  <worldbody>
    <!-- Ground plane extrends infinitely in x and y direction, -->
    <!-- Z is assigned a thickness of 1 but planes are infinitely thin in MuJoCo-->
    <geom name="floor" pos="0 0 -0.5" size="0 0 1" type="plane" material="matplane"/>


    <!-- light source positioned 5 units up and pointing down -->
    <light directional="true" diffuse=".8 .8 .8" specular=".2 .2 .2" pos="0 0 5" dir="0 0 -1"/>

    <!-- sites that fixed in the scene -->
    <site name="s0" pos="-0.15 0 0" size="0.02"/>
    <site name="x0" pos="0 -0.15 0" size="0.02" rgba="0 .7 0 1" group="1"/>

    <body pos="0 0 0">
      <!-- sets upper arm to be 0.5 units long with capsule shape (cylinder with hepisphere ends) -->
      <geom name="upper arm" type="capsule" size="0.045" fromto="0 0 0  0.5 0 0" rgba=".5 .1 .1 1"/>
      <!-- joint named shoulder with default arguments -->
      <joint name="shoulder"/>
      <!-- geom named shoulder to give it a semi-transparent cylinder shape -->
      <geom name="shoulder" type="cylinder" pos="0 0 0" size=".1 .05" rgba=".5 .1 .8 .5" mass="0" group="1"/>

      <!-- sites that are attached to the upper arm -->
      <site name="s1" pos="0.15 0.06 0" size="0.02"/>
      <site name="s2" pos="0.15 -0.06 0" size="0.02"/>
      <site name="s3" pos="0.4 0.06 0" size="0.02"/>
      <site name="s4" pos="0.4 -0.06 0" size="0.02"/>
      <site name="s5" pos="0.25 0.1 0" size="0.02"/>
      <site name="s6" pos="0.25 -0.1 0" size="0.02"/>
      <site name="x1" pos="0.5 -0.15 0" size="0.02" rgba="0 .7 0 1" group="1"/>

      <body pos="0.5 0 0">
        <!-- forearm same as upper arm but starts at 0.5 units along the X axis -->
        <geom name="forearm" type="capsule" size="0.035" fromto="0 0 0  0.5 0 0" rgba=".5 .1 .1 1"/>

        <!-- joint named elbow with default arguments -->
        <joint name="elbow"/>
        <geom name="elbow" type="cylinder" pos="0 0 0" size=".08 .05" rgba=".5 .1 .8 .5" mass="0" group="1"/>

        <!-- sites that are attached to the forearm -->
        <site name="s7" pos="0.11 0.05 0" size="0.02"/>
        <site name="s8" pos="0.11 -0.05 0" size="0.02"/>
      </body>
    </body>
  </worldbody>

  <!-- attachments between the sites -->
  <tendon>
    <spatial name="SF" width="0.01">
      <site site="s0"/>
      <geom geom="shoulder"/>
      <site site="s1"/>
    </spatial>

    <spatial name="SE" width="0.01">
      <site site="s0"/>
      <geom geom="shoulder" sidesite="x0"/>
      <site site="s2"/>
    </spatial>

    <spatial name="EF" width="0.01">
      <site site="s3"/>
      <geom geom="elbow"/>
      <site site="s7"/>
    </spatial>

    <spatial name="EE" width="0.01">
      <site site="s4"/>
      <geom geom="elbow" sidesite="x1"/>
      <site site="s8"/>
    </spatial>

    <spatial name="BF" width="0.009" rgba=".4 .6 .4 1">
      <site site="s0"/>
      <geom geom="shoulder"/>
      <site site="s5"/>
      <geom geom="elbow"/>
      <site site="s7"/>
    </spatial>

    <spatial name="BE" width="0.009" rgba=".4 .6 .4 1">
      <site site="s0"/>
      <geom geom="shoulder" sidesite="x0"/>
      <site site="s6"/>
      <geom geom="elbow" sidesite="x1"/>
      <site site="s8"/>
    </spatial>
  </tendon>

 <!-- allow tendons to have muscles, that can then apply forces to the bodies -->
  <actuator>
    <muscle name="SF" tendon="SF"/>
    <muscle name="SE" tendon="SE"/>
    <muscle name="EF" tendon="EF"/>
    <muscle name="EE" tendon="EE"/>
    <muscle name="BF" tendon="BF"/>
    <muscle name="BE" tendon="BE"/>
  </actuator>
</mujoco>
"""

# Lorn

model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

def muscle_pattern(t):
    punch_time = 2
    if t < punch_time:
        return np.array([
            1.0,  # Shoulder flexor (SF) - fully activated
            0.0,  # Shoulder extensor (SE) - fully relaxed
            1.0,  # Elbow flexor (EF) - fully relaxed
            0.0,  # Elbow extensor (EE) - fully activated
            0.0,  # (BF)
            0.0   # (BE)
        ])
    else:
        return np.array([0] * 6)  # fully relaxed


frames = []
mujoco.mj_resetData(model, data)
with mujoco.Renderer(model) as renderer:

    while data.time < duration:
        data.ctrl[:] = muscle_pattern(data.time)

        mujoco.mj_step(model, data)

        if len(frames) < data.time * framerate:
            renderer.update_scene(data)
            frames.append(renderer.render())

media.show_video(frames, fps=framerate)

This browser does not support the video tag.


## Novel Example

<u>Lorn</u>

The first thing we thought when we saw the tendon arm was "I wonder if we can hit something with it?". The second thing we thought was "how hard?". Unfortunately the tendon arm is kind of wimpy and difficult to control, relying on many individual tendons, carefully balanced to create a realistic-ish arm. However I'm looking for a bit more of a blunt instrument. To hit something harder I'll need an arm with greater mass moving at a higher speed as well as some method to control that. To do this I'm going to get rid of all the tendons, replace the actuators that move the arm with powerful motors, add a big hammer fist on the end, try and get it to punch, and then throw things at it. 

Here is the mujoco model I ended up with. It turns out that by virtue of wanting to hit things really hard and fast, you create a system that simulators really struggle with. Numerical methods are usually accurate for functions that are differentiable (smooth), so when you start slamming things into eachother at high speed over and over again, things get crazy quickly. Even using Newtons method as the solver for the system of equations (I'm not sure if this is just for the ridgid body constraints or for the system of equations in the implicit Euler mujoco uses as well) and an implicit integrator which are both optimized for stiff systems like this, I had to be very careful in my actuations otherwise my arm would just ragdoll-flail. My initial plan (an arm that cocked back, tracked targets as they came in, hit them, and cocked back) was infeasable as I could not return to a stable arm position after a punch. Eventually I ended up with something somewhat crude that satisfied my constraints. I made all the changes I mentioned previously, but instead of a punch, I locked the elbow of the arm and spun the shoulder. 


``` xml
<mujoco model="power hammer :)">
  <!-- Keeping the relativly conservative options as this is a very stiff system -->

  <option timestep="0.005" iterations="50" solver="Newton" tolerance="1e-10"/>

  <asset>
      
    <!-- ...same world materials as previous example... -->

    <!-- Hammer material -->
    <material name="hammer_mat"
              rgba="0.8 0.7 0.2 1"
              specular="0.5"
              shininess="0.8"/>

    <!-- Base material -->
    <material name="base_mat"
              rgba="0.2 0.2 0.2 1"
              specular="0.3"
              shininess="0.6"/>

    <!-- Head material -->
    <material name="head_mat"
              rgba="0.1 0.1 0.1 1"
              specular="0.5"
              shininess="0.8"/>
  </asset>

  <worldbody>
    
    <!-- ...floor and lighting are the same as previous example... -->


    <!-- Base Cylinder -->
    <body name="base" pos="0 0 0">
      <geom type="cylinder" size="0.5 1.5"
            rgba="0.2 0.2 0.2 1" mass="500" material="base_mat"/>

      <!-- Shoulder cap thing on top -->
      <geom type="sphere" size="0.6" pos="0 0 1.5"
            rgba="0.1 0.1 0.1 1" mass="10" material="head_mat"/>

      <!-- Upper Arm -->
      <body name="upper_arm" pos="0 0 1.5">
        <geom type="capsule" fromto="0 0 0 1.2 0 0"
              size="0.12" rgba="0.6 0.2 0.2 1" mass="40"/>
        <joint name="shoulder" type="hinge" axis="0 0 1"
               damping="1"/> <!-- Free spinning -->

        <!-- Forearm -->
        <body name="forearm" pos="1.2 0 0">
          <geom type="capsule" fromto="0 0 0 1.0 0 0"
                size="0.1" rgba="0.6 0.2 0.2 1" mass="30"/>
          <joint name="elbow" type="hinge" axis="0 0 1"
                 range="89 91" stiffness="10000" damping="1000"/> <!-- Locked elbow -->

          <!-- Hammer Fist -->
          <body name="hammer_body" pos="1.0 0 0">
            <geom name="hammer" type="box" size="0.3 0.4 0.3"
                  mass="100" material="hammer_mat"/>
          </body>
        </body>
      </body>
    </body>

    <!-- Target Ball -->
    <body name="target" pos="10 0 10">
      <geom name="target_geom" type="sphere" size="0.3"
            rgba="1 0.2 0.2 1" friction="0.1 0.1 0.01" mass="5"/>
      <freejoint/>
    </body>
  </worldbody>

  <!-- Actuators -->
  <actuator>
    <!-- Powerful motor for shoulder with no control limitations -->
    <motor joint="shoulder" gear="5000" ctrllimited="false"/>
  </actuator>

  <contact>
    <pair geom1="hammer" geom2="target_geom"
          friction="0.8 0.05 0.001" solref="0.01 1"/>
  </contact>
</mujoco>

```

I do think a proper punching arm is feasable. A design that comes to mind looks more like a classical robot arm (the kind you see if you look up mujoco robot arm) with a slider joint that does the actual punching. That way you contain all the unstable motion to a singe joint and are still able to do nice inverse kinematics for target tracking with the robot arm. Really I wanted a punching arm that looked kind of like an anti-aircraft turret on a battleship.

In [9]:
from IPython.display import HTML

HTML("""
<video width="1000" height="600" controls>
  <source src="unstable/sim.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")